In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
wine = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
wine.head()

In [ ]:
from sklearn.model_selection import train_test_split
X = wine.drop(["quality"], axis=1)
y = wine["quality"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, random_state=2019)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=10, random_state=2019)
tree.fit(X_train, y_train)

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(tree, out_file='tree.dot', feature_names=X.columns)
print(open('tree.dot').read())

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = tree.predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
from sklearn.model_selection import GridSearchCV

tree_params = {"max_depth": np.arange(2, 10),
               "min_samples_leaf": np.arange(2, 5),
               "min_samples_split": np.arange(2, 5),
               "max_features": np.arange(1,10)
              }

tree_grid = GridSearchCV(tree, tree_params, cv=5, scoring='accuracy') # cross-valid by 5 blocks
tree_grid.fit(X_train, y_train)

In [ ]:
tree_grid.best_params_

In [ ]:
tree_grid.best_score_

In [ ]:
pd.DataFrame(tree_grid.cv_results_).head().T

best_tree = tree_grid.best_estimator_
y_pred = best_tree.predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
optimal_tree = DecisionTreeClassifier(max_depth = 7, 
                                      max_features = 8, 
                                      min_samples_leaf = 2, 
                                      min_samples_split = 2, 
                                      random_state = 2019)
optimal_tree.fit(X_train, y_train)
y_pred = optimal_tree.predict(X_valid)
print(accuracy_score(y_valid, y_pred))

**Random forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 50, random_state = 2019, max_depth = 6)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_valid)

accuracy_score(y_valid, y_pred)

In [ ]:
forest_params = {'n_estimators': np.arange(1,7),
                 'max_depth': np.arange(2, 8),
                 'min_samples_split': np.arange(2, 6),
                 'min_samples_leaf': np.arange(1,6),
                 'max_features': np.arange(1, 6)}

forest_grid = GridSearchCV(rf, forest_params, cv=5, scoring='accuracy')
forest_grid.fit(X_train, y_train)

In [ ]:
forest_grid.best_params_

In [ ]:
rforest = RandomForestClassifier(n_estimators = 6, 
                                 max_features = 4, 
                                 min_samples_leaf = 5, 
                                 min_samples_split = 2,
                                 random_state = 8, 
                                 max_depth = 6) 
rforest.fit(X_train, y_train)
y_pred = rforest.predict(X_valid)

accuracy_score(y_valid, y_pred)

In [ ]:
rforest.feature_importances_